## PULVAR NOTEBOOK
### FOR PULSAR VARIABILITY ANALYSIS 

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import sys
import pulvar_functions as pvf
import os
from matplotlib.patches import Rectangle
from tqdm import tqdm_notebook
from matplotlib import gridspec
import george
from george import kernels
from scipy.optimize import minimize
import scipy.signal as scisig
import datetime

### ENTER PATH FOR THE INPUT TEXT FILE:

In [ ]:
matrix_file = 'J0738-4042_PARKES_1400.txt'

rawdata = np.loadtxt(matrix_file)

print('This data set contains',rawdata.shape[1],'pulsar observations, each with',rawdata.shape[0]-1,'phase bins.')

### ENTER NAME OF DIRECTORY IN WHICH THINGS WILL BE SAVED:

In [ ]:
directory_name = './J0738-4042_1400MHZ'

if not (os.path.exists('./{0}/'.format(directory_name))):
    os.mkdir('./{0}/'.format(directory_name))

### (OPTIONAL) RUN THIS CELL IF YOU WOULD LIKE TO PROVIDE A LIST OF DAYS ON WHICH OBSERVATIONS SHOULD BE EXCLUDED FROM THE ANALYSIS:

In [ ]:
# LIST OF MJDS TO BE REMOVED:
bad_mjds = [53040, 53118, 53174, 53175, 53193, 53249, 53667, 53971, 57915, 58608, 58645]

bad_mjds_index = []
bad_mjds = np.atleast_1d(bad_mjds)
for b in bad_mjds:
    for i in range(rawdata.shape[1]):
        if abs(rawdata[rawdata.shape[0]-1,i]-b) < 1.0:
            bad_mjds_index.append(i)

raw_before = rawdata.shape[1]
rawdata = np.delete(rawdata,bad_mjds_index,1)
raw_after = rawdata.shape[1]

print(raw_before-raw_after,'observations were manually removed')

### SPLIT THE DATA INTO PROFILES AND MJDS:

In [ ]:
mjd = rawdata[rawdata.shape[0]-1,:]
#Tobs = rawdata[rawdata.shape[0]-2,:]
data = rawdata[0:rawdata.shape[0]-1,:]
bins = data.shape[0]

### (OPTIONAL) RUN CELL TO SAVE PLOTS OF RAW PROFILES:

In [ ]:
pvf.save_profiles(directory_name,data,mjd,data.shape[0],'raw_profile_plots',no_y_lim=True)

### REMOVE PROFILES WITH NOISE ABOVE A CERTAIN THRESHOLD:

In [ ]:
noise_threshold = 2
# (An observation is removed if the standard deviation of the off-pulse region is more than a factor of <noise_threshold> larger than the median value taken from the off-pulse regions across all observations)

baselineremoved, removedprofiles, rmsperepoch, outlierlist, inlierlist = pvf.removebaseline(data, noise_threshold)
mjd_remain = np.delete(mjd,outlierlist)
mjdremoved = np.delete(mjd,inlierlist)

### (OPTIONAL) SAVE PLOTS OF PROFILES THAT WERE REMOVED FOR BEING TOO NOISY:

In [ ]:
try:
    pvf.save_profiles(directory_name,removedprofiles,mjdremoved,data.shape[0],'noisy_profile_plots')
except:
    print('Noisy profile removal was not performed. No plots to save.')

### VIEW THE BRIGHTEST PROFILE IN THE DATA SET:

In [ ]:
brightest_profile = pvf.findbrightestprofile(baselineremoved,rmsperepoch)
pvf.plot_profile(data[:,brightest_profile]);

### IF S/N IS TOO LOW, RESAMPLE THE DATA TO INCREASE IT:

In [ ]:
# SET THE THRESHOLD:
s_n_threshold = 2

brightprofpeak = np.max(baselineremoved[:,brightest_profile])
brightprofrms = rmsperepoch[brightest_profile]

if brightprofpeak/brightprofrms < s_n_threshold :
    print('Resampling...')
    resampled = scisig.resample(baselineremoved,int(bins/8))
else:
    print('Resampling not necessary.')
    resampled = baselineremoved
baselineremoved = resampled

### CROSS CORRELATE THE PROFILES TO ALIGN THEM AND ALSO NORMALISE PEAKS TO 1.0:

In [ ]:
aligned_data, template = pvf.aligndata(baselineremoved, brightest_profile, directory_name)
originaltemplate = np.copy(template)
originaltemplate_normalised = originaltemplate/(np.max(originaltemplate))

### PLOT CURRENT MEDIAN PROFILE (PEAK POSITIONED AT PERIOD / 4):

In [ ]:
ax = pvf.plot_profile(originaltemplate_normalised)
ax.set_ylabel('Normalised Flux Density');

### CHOOSE THE 'BEGIN' AND 'END' BINS FOR YOUR PULSE WINDOW BASED ON THE ABOVE PROFILE:

In [ ]:
window_begin = 140
window_end = 340

### CHOOSE THE WINDOWS FOR ANY OTHER PULSE REGIONS ([BEGIN_1,END_1,BEGIN_2,END_2,...]):

In [ ]:
other_pulse_regions = []

### PLOT TO SEE IF CHOSEN WINDOWS ARE SATISFACTORY (IF NOT EDIT CELLS ABOVE):

In [ ]:
ax = pvf.plot_profile(originaltemplate_normalised)
plt.vlines(window_begin,-0.1,1.1,linestyles='dashed',color='k')
plt.vlines(window_end,-0.1,1.1,linestyles='dashed',color='k')
ax.add_patch(Rectangle((window_begin,-0.1),(window_end-window_begin),1.2,facecolor='green',alpha = 0.1, zorder = 39))
for r in range(int(len(other_pulse_regions)/2)):
    plt.vlines(other_pulse_regions[2*r],-0.1,1.1,linestyles='dashed')
    plt.vlines(other_pulse_regions[2*r+1],-0.1,1.1,linestyles='dashed')
    ax.add_patch(Rectangle((other_pulse_regions[2*r],-0.1),(other_pulse_regions[2*r+1]-other_pulse_regions[2*r]),1.2,facecolor='darkorange',alpha = 0.1, zorder = 39))

ax.set_ylabel('Normalised Flux Density');

### RE-ALIGN AND RE-NORMALISE PROFILES BY MAXIMISING NUMBER OF STABLE BINS:

In [ ]:
# TAKES A FEW SECONDS PER OBSERVATION

aligned_data_norm, aligned_data = pvf.smart_align(originaltemplate_normalised,baselineremoved[:,:])

### CALCULATE THE STD ACROSS ALL PROFILES IN THE OFF-PULSE (NON-COLOURED) REGIONS IN PLOT ABOVE:

In [ ]:
on_pulse_bins = []

for a in range(window_end-window_begin+1):
    on_pulse_bins.append(window_begin+a)
for b in range(int(len(other_pulse_regions)/2)):
    for c in range(other_pulse_regions[2*b+1]-other_pulse_regions[2*b]+1):
        on_pulse_bins.append(other_pulse_regions[2*b]+c)

off_pulse_data = np.delete(aligned_data_norm,on_pulse_bins,0)

rms = np.std(off_pulse_data)

### VIEW ALL ALIGNED AND NORMALISED PROFILES IN PULSE WINDOW (WITH RED MEDIAN PROFILE):

In [ ]:
post_bin_match_median = np.zeros((aligned_data_norm.shape[0]))
for b in range(post_bin_match_median.shape[0]):
    post_bin_match_median[b] = np.median(aligned_data_norm[b,:])

ax = pvf.plot_profile(aligned_data_norm[window_begin:window_end,:])
ax.plot(post_bin_match_median[window_begin:window_end],'r--',linewidth=4)
ax.set_ylabel('Normalised Flux Density');

### (OPTIONAL) SAVE PLOTS OF ALL SURVIVING INDIVIDUAL PROFILES:

In [ ]:
pvf.save_profiles(directory_name,aligned_data_norm[window_begin:window_end,:],mjd_remain,data.shape[0],'pulse_window_plots','Normalised Flux Density',template=post_bin_match_median[window_begin:window_end])

### FIT A GAUSSIAN PROCESS MODEL TO EACH PHASE BIN IN THE PULSE WINDOW:

#### SELECT A GAUSSIAN PROCESS LENGTH SCALE IN DAYS:

In [ ]:
gp_len_scale = 110

#### FIT THE MODEL:

In [ ]:
gp_data = np.zeros((window_end-window_begin,int(mjd_remain[-1])-int(mjd_remain[0]))) 
gp_var = np.zeros((window_end-window_begin,int(mjd_remain[-1])-int(mjd_remain[0])))
all_profile_residuals = np.zeros((window_end-window_begin,aligned_data_norm.shape[1]))
gp_mjds = np.zeros((int(mjd_remain[-1])-int(mjd_remain[0])))
x_pred = np.arange(int(mjd_remain[0]), int(mjd_remain[-1]),1)
all_noise = np.zeros((window_end-window_begin,aligned_data_norm.shape[1]))

for g in tqdm_notebook(range(window_end-window_begin)):
    y_profile_residuals = (aligned_data_norm[window_begin+g,:] - post_bin_match_median[window_begin+g])/rms
    y_noise = np.std(y_profile_residuals)* np.ones_like(mjd_remain)    
    kernel = np.var(y_profile_residuals) * kernels.ExpSquaredKernel(gp_len_scale)
    gp = george.GP(kernel)
    gp.compute(mjd_remain, y_noise)
    pred, pred_var = gp.predict(y_profile_residuals, x_pred, return_var=True)

    def neg_ln_like(p):
        gp.set_parameter_vector(p)
        return -gp.log_likelihood(y_profile_residuals)

    def grad_neg_ln_like(p):
        gp.set_parameter_vector(p)
        return -gp.grad_log_likelihood(y_profile_residuals)

    result = minimize(neg_ln_like, gp.get_parameter_vector(), jac=grad_neg_ln_like)
    gp.set_parameter_vector(result.x)
    
    pred, pred_var = gp.predict(y_profile_residuals, x_pred, return_var=True)
    
    
    gp_data[g,:] = pred 
    gp_var[g,:] = pred_var
    all_profile_residuals[g,:] = y_profile_residuals
    all_noise[g,:] = y_noise

gp_mjds[:] = x_pred

### (OPTIONAL) VIEW EACH THE GAUSSIAN PROCESS MODELS:

In [ ]:
for i in tqdm_notebook(range(gp_data.shape[0])):
    print('PHASE BIN:',i)
    fig = plt.figure(figsize=(15,4))
    gs = gridspec.GridSpec(2,1)
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(post_bin_match_median[window_begin:window_end],'k')
    plt.vlines(i,-np.max(post_bin_match_median[window_begin:window_end])*0.05,np.max(post_bin_match_median[window_begin:window_end])*1.05,'r',linewidth=4,alpha=0.2)
    ax1.set_xlim(0-1,post_bin_match_median[window_begin:window_end].shape[0])
    ax1.set_ylim(-np.max(post_bin_match_median[window_begin:window_end])*0.05,np.max(post_bin_match_median[window_begin:window_end])*1.05)
    plt.tick_params(axis='y', which='both', left=True, labelleft=False)
    ax1.tick_params(axis='x', which='both', bottom=False, labelbottom=False)
    plt.grid()
    
    ax2 = fig.add_subplot(gs[1, 0])
    fig.subplots_adjust(hspace=0.0)
    ax2.plot(gp_mjds[:],gp_data[i,:],'k')
    ax2.errorbar(mjd_remain, all_profile_residuals[i,:], yerr=all_noise[i,:], fmt=".k", capsize=0);
    ax2.fill_between(gp_mjds[:], gp_data[i,:] - np.sqrt(gp_var[i,:]), gp_data[i,:] + np.sqrt(gp_var[i,:]),color="k", alpha=0.2)
    ax2.set_xlim(gp_mjds[0],gp_mjds[-1])
    plot_y_min = np.minimum(np.min(gp_data[:,:] - np.sqrt(gp_var[:,:])),np.min(all_profile_residuals[:,:]-all_noise[:,:]))
    plot_y_max = np.maximum(np.max(gp_data[:,:] + np.sqrt(gp_var[:,:])), np.max(all_profile_residuals[:,:]+all_noise[:,:]))
    ax2.set_ylim(plot_y_min,plot_y_max)
    ax1.tick_params(axis='x', size=20)
    plt.grid()
    plt.show()
    plt.close(fig)

### CALCULATE PEAK FLUX DENSITY (NON-NORMALISED) ACROSS THE DATA SET:

In [ ]:
peak_flux_den = np.zeros((aligned_data.shape[1]))

for ad in range(aligned_data.shape[1]):
    peak_flux_den[ad] = np.max(aligned_data[:,ad])

peak_flux_den = peak_flux_den

### PLOT PULSAR VARIABILITY MAP:

In [ ]:
pvf.variability_map_plot(mjd_remain,gp_data,peak_flux_den,post_bin_match_median,window_begin, window_end,data.shape[0]) 
print('- Date of observations are shown on the variability map as verticle lines.\n')
print('- The units of the variability map is:\n  The standard deviation of all off-pulse phase bins across all observations kept in the analysis.')